In [1]:
# import
!pip install jiwer

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# import
import os
import torch
from jiwer import wer
import soundfile as sf
from datasets import load_dataset,load_from_disk
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

/home/haram/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 에러 로깅 가능
os.environ['CUDA_LAUNCH_BLOCKING'] = '3'

# cuda가 볼 수 잇는 GPU => 내가 사용할 GPU
os.environ["CUDA_VISIBLE_DEVICES"] = '4,5,6,7'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model_name = "facebook/wav2vec2-base-960h"
model_name="facebook/wav2vec2-large-robust-ft-libri-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name).to(device)

ds = load_from_disk("./data/exist_test/dict2")

print(ds)
test_ds = ds['test1']

def map_to_array(batch):
    speech, _ = sf.read(batch["path"])
    batch["speech"] = speech
    return batch

test_ds = test_ds.map(map_to_array)

def map_to_pred(batch):
    inputs = processor(batch["speech"], sampling_rate=16000, return_tensors="pt", padding="longest")
    input_values = inputs.input_values.to(device)
    
    model.eval()
    
    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    batch["result"] = transcription

    print(batch["transcription"]," ",transcription)
    print()
    return batch

result = test_ds.map(map_to_pred, batched=True, batch_size=16)

print(len(result))
print(type(result))

total  = len(result)
correct = 0
for data in result:
    if data["transcription"] == data["result"]:
        correct+=1

accuracy = (correct/total)*100
print("WER:", wer(result["transcription"], result["result"]))
accuracy

Some weights of the model checkpoint at facebook/wav2vec2-large-robust-ft-libri-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-robust-ft-libri-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You

DatasetDict({
    test1: Dataset({
        features: ['path', 'transcription', 'audio'],
        num_rows: 200
    })
    train2: Dataset({
        features: ['path', 'transcription', 'audio'],
        num_rows: 200
    })
})


Parameter 'function'=<function map_to_pred at 0x7f8b42ac8a60> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map:   0%|          | 0/200 [00:00<?, ? examples/s]2024-04-20 11:13:42.728803: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-20 11:13:42.764201: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: At

['BUT THEN YOU SEE I HAVE BEEN MAKING TOYS FOR HUNDREDS OF YEARS AND I MAKE SO MANY IT IS NO WONDER I AM SKILLFUL AND NOW IF YOU ARE READY TO GO HOME I LL HITCH UP THE REINDEER AND TAKE YOU BACK AGAIN', 'AFTER MOTHER HUBBARD HAD GIVEN ME A GOOD DINNER AND I HAD EATEN SOME OF THE MOST DELICIOUS CLOVER I HAVE EVER TASTED SANTA TOOK ME UP INTO HIS WORK ROOM AND SAT ME UPON THE TABLE IF I CAN ONLY MAKE RABBITS HALF AS NICE AS YOU ARE HE SAID', 'SO AFTER MOTHER HUBBARD HAD GIVEN ME A GOOD MEAL OF TURNIPS AND SLICED CABBAGE SANTA CLAUS PUT THE MAGIC COLLAR AROUND MY NECK AND I STARTED FOR HOME I TOOK MY TIME ON THE JOURNEY FOR I KNEW NOTHING COULD HARM ME', 'AND SO HE SNIPPED A LITTLE OFF THE FUR HE WAS CUTTING', 'OH NO SAID I I PREFER TO RUN BY MYSELF FOR I CAN EASILY FIND THE WAY AND I WANT TO SEE THE COUNTRY IF THAT IS THE CASE REPLIED SANTA I MUST GIVE YOU A MAGIC COLLAR TO WEAR SO THAT YOU WILL COME TO NO HARM', 'AFTER I GOT HOME REPLIED THE RABBIT THE COLLAR DISAPPEARED FROM AROUND MY 

Map:  16%|█▌        | 32/200 [00:07<00:35,  4.77 examples/s]

...

Map: 100%|██████████| 200/200 [00:32<00:00,  6.75 examples/s]

['WERE NEARLY ALWAYS SORRY FOR IT AFTERWARDS PRESENTLY SHE SPIED CHEE CHEE SWINGING THROUGH THE TREES STILL LOOKING FOR THE DOCTOR WHEN CHEE CHEE SAW HER HE CAME INTO HER TREE', 'AND BEGAN READING THE FAIRY STORIES TO HIMSELF CHEE CHEE AND POLYNESIA WATCHED HIM KEEPING VERY QUIET AND STILL', 'THE QUEEN OF THE FAIRIES THAT SPEAK TO THEE I AM HIDING IN A ROSE BUD OH TELL ME FAIRY QUEEN CRIED BUMPO CLASPING HIS HANDS IN JOY', "HE CLIMBED HIGH TREES AND LOOKED OUT FROM THE TOP BRANCHES TO TRY AND SEE THE DOCTOR'S HIGH HAT HE WAVED AND SHOUTED HE CALLED TO ALL THE ANIMALS BY NAME BUT IT WAS NO USE", 'SO YOU ARE CAUGHT AGAIN THIS TIME YOU SHALL NOT ESCAPE TAKE THEM ALL BACK TO PRISON AND PUT DOUBLE LOCKS ON THE DOOR', 'THEY STUMBLED INTO WET BOGGY PLACES THEY GOT ALL TANGLED UP IN THICK CONVOLVULUS RUNNERS THEY SCRATCHED THEMSELVES ON THORNS AND TWICE THEY NEARLY', 'HE MUST BEGIN SCRUBBING THE KITCHEN FLOOR THEY WERE ALL VERY UNHAPPY THIS IS A GREAT NUISANCE SAID THE DOCTOR', 'IT MEANT THAT 

Map: 100%|██████████| 200/200 [00:32<00:00,  6.18 examples/s]


200
<class 'datasets.arrow_dataset.Dataset'>
WER: 0.02188054405677114


56.99999999999999

In [4]:
# # 에러 로깅 가능
# os.environ['CUDA_LAUNCH_BLOCKING'] = '3'

# # cuda가 볼 수 잇는 GPU => 내가 사용할 GPU
# os.environ["CUDA_VISIBLE_DEVICES"] = '4,5,6,7'
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# dataset_name= "test1"
# model_name = "facebook/wav2vec2-base-960h"
# dataset_dir ="./data/exist_test/dict2"

# processor = Wav2Vec2Processor.from_pretrained(model_name)
# model = Wav2Vec2ForCTC.from_pretrained(model_name).to(device)

# eval_prev_train_model(model_name,dataset_dir,device,dataset_name)